## BioPAX abstraction: Pathway-centered view

TODO : 
- find pathway examples from Reactome
- define abstraction rules
- implement in SPARQL

![image](../Schemas/Screenshot%20from%202025-02-07%2011-09-05.png)
![image](../Schemas/Screenshot%20from%202025-02-07%2011-09-28.png)

### First exploration on Reactome v91 (Homo Sapiens)

Total number of pathways : 2751

- How many pathways have sub-pathways ?

```sparql
SELECT (COUNT (DISTINCT ?pathway1) AS ?nbPathwayRoot) 
WHERE {
  ?pathway1 rdf:type bp3:Pathway .
  ?pathway2 rdf:type bp3:Pathway .
  
  ?pathway1 bp3:pathwayComponent ?pathway2 .
}
```

808 pathways (~ 29% of pathways have sub-pathways)

- How many pathways can be relied to their subpathways via bp3:nextStep ?

```sparql
SELECT (COUNT (DISTINCT ?pathwayRoot) AS ?nbPathwayRoot) 
WHERE {
  ?pathwayRoot rdf:type bp3:Pathway .
  ?subPathway rdf:type bp3:Pathway .
  
  ?pathwayRoot bp3:pathwayComponent ?subPathway .
  
  ?pathwayRoot bp3:pathwayOrder ?pathwayRootStep .
  ?subPathway bp3:pathwayOrder ?subPathwayStep .
  
  ?pathwayRootStep (bp3:nextStep)+ ?subPathwayStep .
}
```

148 pathways 

- How many sub-pathways have an entering nextStep from another pathway or a leaving nextStep to another pathway ? 

```sparql
SELECT (COUNT(DISTINCT ?subPathway) AS ?nbSubPathway) 
WHERE {
  ?pathwayRoot rdf:type bp3:Pathway .
  ?subPathway rdf:type bp3:Pathway .
  
  ?pathwayRoot bp3:pathwayComponent ?subPathway .
  
  FILTER EXISTS {
    {
      ?enteringNextStep bp3:nextStep ?subPathwayStep .
      ?previousPathway (bp3:pathwayOrder)* ?enteringNextStep .
    } 
    UNION 
    {
      ?subPathwayStep bp3:nextStep ?leavingNextStep .
      ?nextPathway (bp3:pathwayOrder)* ?leavingNextStep .
    }
  }
}
```



- Are there Biochemical Reactions that are uniquely linked to the root pathway via a bp3:pathwayComponent relationship and not via a bp3:PathwayStep ?

```sparql
SELECT ?pathwayRoot (COUNT(DISTINCT ?pathwayRootDirectProcess) AS ?nbPathwayRootDirectProcess)
WHERE {
  ?pathwayRoot rdf:type bp3:Pathway ;
               bp3:pathwayComponent ?subPathway, ?pathwayRootDirectProcess .
  
  ?subPathway rdf:type bp3:Pathway ;
              bp3:pathwayOrder ?subPathwayStep .
  
  ?subPathwayStep bp3:stepProcess ?subPathwayStepProcess .
  
  ?pathwayRootDirectProcess rdf:type bp3:BiochemicalReaction .
  
  FILTER NOT EXISTS {
    ?subPathwayStepProcess rdf:type bp3:BiochemicalReaction .
    FILTER (?pathwayRootDirectProcess = ?subPathwayStepProcess)
  }
}
GROUP BY ?pathwayRoot 
ORDER BY DESC(?nbPathwayRootDirectProcess)
```

Max 52 Biochemical reactions for a Pathway

This situation occurs for 311 pathways

~ 8000 nextSteps entre différents pathways


### SPARQL queries for abstraction

#### 1 - SuperPathwayOf

Defines a relation that decribes the direct hierarchy of pathways.

In [ ]:
query_super_pathway_of = """
CONSTRUCT {
  ?pathway abstraction:SuperPathwayOf ?subPathway .
}
WHERE {
  ?pathway rdf:type bp3:Pathway .
  
  ?pathway bp3:pathwayComponent ?subPathway .
  ?subPathway rdf:type bp3:Pathway .
}
"""

#### 2 - NextStepPathway

Defines a relation that describes the sequence of pathway steps across different pathways

In [1]:
query_next_step_pathway = """ 
CONSTRUCT {
  ?pathway abstraction:NextStepPathway ?nextPathway
}
WHERE {
  ?pathway rdf:type bp3:Pathway .
  ?nextPathway rdf:type bp3:Pathway .
  
  ?pathway bp3:pathwayOrder ?pathwayStep .
  
  ?pathwayStep bp3:nextStep ?nextStep .
  
  ?nextPathway bp3:pathwayOrder ?nextStep .
  
  FILTER (?pathway != ?nextPathway)
  FILTER NOT EXISTS {
    ?pathwayRoot bp3:pathwayComponent* ?pathway .
    ?pathwayRoot bp3:pathwayComponent* ?nextPathway .
  }
}
"""

#### 3 - InnerNextStepPathway 

Defines a relation that describes the sequence of pathway steps belonging to the same pathway

In [ ]:
query_inner_next_step_pathway = """ 
CONSTRUCT {
  ?pathway abstraction:InnerNextStepPathway ?nextPathway
}
WHERE {
  ?pathway rdf:type bp3:Pathway .
  ?nextPathway rdf:type bp3:Pathway .
  
  ?pathwayRoot bp3:pathwayComponent* ?pathway .
  ?pathwayRoot bp3:pathwayComponent* ?nextPathway .
  
  ?pathway bp3:pathwayOrder ?pathwayStep .
  
  ?pathwayStep bp3:nextStep ?nextStep .
  
  ?nextPathway bp3:pathwayOrder ?nextStep .
  
  FILTER (?pathway != ?nextPathway)
}
"""